# Notebook 00: CHS 核心理念与仿真引擎

欢迎来到CHS（水文模拟系统）的世界！本系列教程旨在引导您掌握CHS SDK的强大功能。

作为所有后续教程的基础，本篇将向您介绍CHS最核心的设计哲学：**代码即Simulink**，并带您动手运行第一个最简单的仿真案例。

## “代码即Simulink”的设计哲学

Simulink是一款强大的图形化仿真工具，用户通过拖拽“模块”并用“连线”将它们连接起来，构建复杂的系统模型。CHS借鉴了这一思想，但将其完全代码化，带来了更高的灵活性和可扩展性。

在CHS中：
- **模块 (Blocks)** 对应于 `config` 字典中的 **`components`**。每一个`component`都是一个Python对象，代表系统中的一个实体，如水库、河道、水泵，甚至是虚拟的扰动信号。
- **连线 (Connections)** 对应于 `config` 字典中的 **`connections`**。每一条`connection`定义了数据如何从一个`component`的输出传递到另一个`component`的输入。
- **求解器 (Solver)** 由 **`SimulationManager`** 类来承担。它负责解析您的`config`配置，构建系统，并按照设定的时间步长（`dt`）和总时长（`total_time`）来驱动整个仿真过程。

通过一个简单的Python字典（`config`）来描述整个系统，我们实现了“代码即文档”和“代码即模型”的目标。这种方式不仅清晰易读，而且极易通过代码进行版本的控制、修改和自动化测试。

## 第一个仿真：水库开环响应

接下来，我们将通过一个最简单的例子——“水库开环仿真”——来直观感受CHS的工作流程。在这个例子中，我们将创建一个水库，并为其提供一个恒定的入流和出流，观察其水位的变化。

请运行下方的代码单元格，查看仿真结果。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from water_system_sdk.src.water_system_simulator.simulation_manager import SimulationManager

# 1. 定义仿真配置字典 (config)
# 这就是我们的“模型图纸”，描述了系统的所有组成部分和它们之间的关系。
config = {
    # 仿真参数：定义仿真的时间和步长
    "simulation_params": {
        "total_time": 200,  # 总仿真时长 (秒)
        "dt": 1             # 时间步长 (秒)
    },
    # 组件：定义系统中的所有“模块”
    "components": {
        # 一个恒定的10m³/s的入流信号
        "inflow_dist": {
            "type": "Disturbance",
            "params": {
                "signal_type": "constant",
                "value": 10.0
            }
        },
        # 一个恒定的5m³/s的出流信号
        "outflow_dist": {
            "type": "Disturbance",
            "params": {
                "signal_type": "constant",
                "value": 5.0
            }
        },
        # 我们的核心研究对象：一个水库模型
        "reservoir1": {
            "type": "ReservoirModel",
            "params": {
                "area": 100.0,       # 水库横截面积 (m²)
                "initial_level": 5.0 # 初始水位 (m)
            }
        }
    },
    # 连接：定义组件之间的“连线”
    "connections": [
        {
            "source": "inflow_dist.output",   # 源：入流信号的输出
            "target": "reservoir1.input.inflow" # 目标：水库模型的入流输入
        },
        {
            "source": "outflow_dist.output",  # 源：出流信号的输出
            "target": "reservoir1.input.outflow"# 目标：水库模型的出流输入
        }
    ],
    # 执行顺序：定义每个时间步中，组件的计算顺序
    "execution_order": ["inflow_dist", "outflow_dist", "reservoir1"],
    # 日志记录器：指定需要记录和输出哪些变量
    "logger_config": [
        "reservoir1.state.level",
        "inflow_dist.output",
        "outflow_dist.output"
    ]
}

# 2. 初始化并运行仿真
# SimulationManager是我们的“仿真引擎”，它会解析config并执行仿真
manager = SimulationManager()
results_df = manager.run(config)

# 3. 结果可视化
plt.figure(figsize=(12, 7))
plt.plot(results_df['time'], results_df['reservoir1.state.level'], label='水库水位 (m)', color='b', linewidth=2)
plt.title('水库开环响应仿真', fontsize=16)
plt.xlabel('时间 (s)', fontsize=12)
plt.ylabel('水位 (m)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

# 4. 打印最终结果以供验证
print("仿真结果预览:")
print(results_df.tail())

# 简单验证
final_level = results_df['reservoir1.state.level'].iloc[-1]
# 理论计算: 净流入 = 10 - 5 = 5 m³/s. 水位上升速率 = 5 / 100 = 0.05 m/s.
# 200秒后水位上升 = 200 * 0.05 = 10 m. 最终水位 = 5 + 10 = 15 m.
expected_final_level = 15.0
print(f"\n最终水位 (仿真值): {final_level:.2f} m")
print(f"最终水位 (理论值): {expected_final_level:.2f} m")


## 结果分析与互动

从上方的图表和输出中，我们可以清晰地看到：
- 水库的水位随着时间的推移线性上升。
- 仿真的最终水位与我们手动计算的理论值（15.0米）完全一致，验证了模型和仿真引擎的正确性。

**现在轮到你了！**

尝试修改上方代码单元格中的`config`参数，然后重新运行它，观察结果的变化。例如：

1.  **让水位下降**：将`outflow_dist`的`value`改为`12.0`，使其大于入流量。
2.  **加快水位变化**：将`reservoir1`的`area`（面积）改小，例如`50.0`。
3.  **延长仿真时间**：将`simulation_params`的`total_time`改为`500`。

通过亲手修改参数并立即看到结果，您将能更深刻地理解CHS的工作方式。在下一个教程中，我们将开始探索更复杂的组件和功能。